In [1]:
import sys, os

project_dir = "/serialize/"
sys.path.append(project_dir)
os.environ["DJANGO_SETTINGS_MODULE"] = "my_project.settings"
# os.environ.setdefault("DJANGO_SETTINGS_MODULE", __file__)

import django
from django.db.models import Count, Q

django.setup()

from serialize.models import SerializeMaster
import sys
import asyncio

In [2]:

# total_parts = SerializeMaster.objects.count()
# print(f"{total_parts=}")

In [3]:

# total_not_scrapped = (
#     SerializeMaster.objects.filter(Q(scrapped="NULL")).values().count()
#     # SerializeMaster.objects.filter(scrapped__exact="NULL").values().count()
# )
# print(f"{total_not_scrapped=}")

In [5]:
async def get_not_scrapped_by_job():
    not_scrapped_by_job = (
        await SerializeMaster.objects.filter(Q(scrapped="NULL"))
        .values("job_number")
        .annotate(qty_parts=Count("part_id"))
        # SerializeMaster.objects.filter(scrapped__exact="NULL").values().count()
    )
    # return not_scrapped_by_job
    print(f"{not_scrapped_by_job=}")
# not_scrapped_by_job = await get_not_scrapped_by_job()

loop = asyncio.get_event_loop()
loop.run_until_complete(get_not_scrapped_by_job())
loop.close()

RuntimeError: This event loop is already running